In [2]:
from docx import Document as DocxDocument
from langchain.docstore.document import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
import pickle, os
from datetime import datetime
from langchain_openai import ChatOpenAI
from langchain.chains.retrieval_qa.base import RetrievalQA
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from pathlib import Path
from __future__ import annotations
from datetime import datetime
from pathlib import Path
from typing import List, Set, Deque, Tuple
from collections import deque
from urllib.parse import urljoin, urlparse
import time, re, xml.etree.ElementTree as ET
import re, requests, html
from langchain.schema import Document
from urllib.parse import urljoin, urlparse
from tqdm import tqdm
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
load_dotenv()

True

In [3]:
docs = []
chunks = []
CHUNK_TOKENS   = 100
CHUNK_OVERLAP  = 10
SEPARATORS     = ["\n\n", "\n", ". ", " "]

splitter = RecursiveCharacterTextSplitter(
    chunk_size      = CHUNK_TOKENS * 6,   # ~400 tokens ≈ 2400 chars
    chunk_overlap   = CHUNK_OVERLAP * 6,  # keep ~40‑token context
    separators      = SEPARATORS,
)

In [4]:
urls = [
    "https://lxera.net/"
]

In [5]:
def _requests_retry_session(retries: int = 3, backoff: int = 1) -> requests.Session:
    from requests.adapters import HTTPAdapter, Retry
    retry = Retry(
        total=retries, backoff_factor=backoff,
        status_forcelist=[429, 500, 502, 503, 504, 403]
    )
    sess = requests.Session()
    sess.mount("https://", HTTPAdapter(max_retries=retry))
    sess.mount("http://",  HTTPAdapter(max_retries=retry))
    return sess

def _clean_text(txt: str) -> str:
    txt = html.unescape(txt)
    txt = txt.replace("\xa0", " ")              # NB‑spaces → normal
    txt = re.sub(r"\n{3,}", "\n\n", txt)        # squeeze vertical space
    txt = re.sub(r"[ \t]{2,}", " ", txt)        # squeeze horizontal
    return txt.strip()

def _extract_with_trafilatura(url: str, html_text: str) -> str | None:
    try:
        import trafilatura
        return trafilatura.extract(html_text, url=url, include_comments=False,
                                   favor_recall=False, include_tables=False)
    except ImportError:
        return None

In [6]:
def load_single_page(url: str, session: requests.Session) -> List[Document]:
    """
    Fetch `url`, extract the main text, clean & split it,
    and return a list[Document] that’s ready for embedding.

    NOTE: The caller must pass in an *already‑configured* requests.Session
    (e.g. with retries and headers) so we don’t create a new TCP connection
    for every page.
    """
    HEADERS = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/125.0.0.0 Safari/537.36"
        ),
        "Accept-Language": "en-US,en;q=0.8,ar;q=0.7",
        "Referer": "https://www.google.com",
    }

    # ➊ use the session that was passed in
    resp = session.get(url, headers=HEADERS, timeout=15)
    if resp.status_code != 200:
        raise RuntimeError(f"Failed to load {url} – status {resp.status_code}")

    html_text = resp.text

    # ➋ same extraction / cleaning logic as before -------------
    main_text = _extract_with_trafilatura(url, html_text)
    if not main_text:
        soup = BeautifulSoup(html_text, "lxml")
        for tag in soup(["script", "style", "nav", "footer",
                         "header", "noscript"]):
            tag.decompose()
        main_text = soup.get_text(separator="\n")

    main_text = _clean_text(main_text)
    if len(main_text) < 200:
        raise ValueError("Content too short after cleaning")

    soup_head = BeautifulSoup(html_text, "lxml").find("head")
    title = (soup_head.title.get_text(strip=True)
             if soup_head and soup_head.title else "")
    canonical = soup_head.find("link", rel="canonical")
    canonical_url = canonical["href"] if canonical else url

    chunks = splitter.split_text(main_text)
    ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    docs: List[Document] = [
        Document(
            page_content=chunk,
            metadata={
                "source": canonical_url,
                "title": title,
                "chunk_index": i,
                "uploaded_at": ts,
            },
        )
        for i, chunk in enumerate(chunks)
    ]
    return docs


In [26]:
load_single_page(    "https://lxera.net/", _requests_retry_session()
)

RuntimeError: Failed to load https://lxera.net/ – status 404

In [7]:
# ───── 2. sitemap helper ───────────────────────────────────────────────────── #
def _discover_sitemap_urls(base_url: str, session: requests.Session) -> List[str]:
    candidates = ["/sitemap.xml", "/sitemap_index.xml", "/sitemap-index.xml"]
    urls = []
    for path in candidates:
        try:
            resp = session.get(urljoin(base_url, path), timeout=5)
            if resp.status_code == 200 and "<urlset" in resp.text:
                tree = ET.fromstring(resp.text)
                urls = [loc.text.strip() for loc in tree.iter("{*}loc")]
                if urls:
                    break
        except Exception:
            pass
    return urls

# ───── 3. link‑scraper for fallback crawl ─────────────────────────────────── #
def _extract_internal_links(html_text: str, base_url: str) -> Set[str]:
    soup = BeautifulSoup(html_text, "lxml")
    links: Set[str] = set()
    for a in soup.select("a[href]"):
        href = a["href"]
        if href.startswith("#") or href.lower().startswith(("mailto:", "tel:")):
            continue
        absolute = urljoin(base_url, href)
        if urlparse(absolute).netloc == urlparse(base_url).netloc:
            links.add(absolute.split("#")[0])
    return links

# ───── 4. site‑wide loader ─────────────────────────────────────────────────── #
def load_site(base_url: str,
              max_pages: int = 100,
              max_depth: int = 3,
              delay: float = 1.0) -> List[Document]:
    """
    Crawl `base_url` (same domain only) and return RAG‑ready Document chunks
    for every page encountered.
    """
    session = _requests_retry_session()
    visited: Set[str] = set()
    docs: List[Document] = []

    # 1️⃣  try sitemap
    to_visit: Deque[Tuple[str, int]] = deque()
    sitemap_urls = _discover_sitemap_urls(base_url, session)
    if sitemap_urls:
        to_visit.extend((u, 0) for u in sitemap_urls)
    else:
        to_visit.append((base_url, 0))

    while to_visit and len(visited) < max_pages:
        url, depth = to_visit.popleft()
        if url in visited or depth > max_depth:
            continue
        visited.add(url)

        try:
            # fetch raw HTML once (so we reuse for link extraction + content parse)
            resp = session.get(url, headers={
                "User-Agent": (
                    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                    "AppleWebKit/537.36 (KHTML, like Gecko) "
                    "Chrome/125.0.0.0 Safari/537.36"
                )
            }, timeout=15)
            if resp.status_code != 200:
                continue

            # 2️⃣  parse & chunk this page
            docs.extend(load_single_page(url, session))

            # 3️⃣  collect more links if we’re not using sitemap
            if not sitemap_urls and depth < max_depth:
                new_links = _extract_internal_links(resp.text, base_url)
                for link in new_links:
                    if link not in visited:
                        to_visit.append((link, depth + 1))

            time.sleep(delay)  # politeness
        except Exception as e:
            print(f"[WARN] {url} skipped – {e}")

        if len(visited) >= max_pages:
            print(f"[INFO] Reached max_pages={max_pages}")
            break

    return docs

In [8]:
session = _requests_retry_session()
for url in urls:
    try:
        print(f"Loading {url} ...")
        docs.extend(load_site(url))
        print(f"Loaded {len(docs)} documents from {url}")
    except Exception as e:
        print(f"Error loading {url}: {e}")
chunks.extend(splitter.split_documents(docs))

Loading https://lxera.net/ ...
Loaded 32 documents from https://lxera.net/


In [9]:
chunks

[Document(page_content='الإبتكار في إدارة التعلم الرقمي\nLxera نظام تعليمي مبتكر وآمن، يمكّن جميع أطراف العملية التعليمية من بناء بيئات تعلم مخصصة وتفاعلية ترتكز على المتعلم، وتدعم التعاون والتقدم الرقمي.\nمزايا النظام\nصمم نظام Lxera ليمنح المؤسسات المُستفيدة أدوات متقدمة ضمن تجربة جديدة مبتكرة وقابلة للتخصيص حسب احتياج المؤسسة للمزايا التالية\nمساعد ذكي مدعوم بالذكاء الاصطناعي\nشات بوت ذكي ومدرب بالكامل على النظام، يقدّم المساعدة الفورية ويجيب على جميع استفسارات المستخدمين بدقة وفعالية.\nواجهة مستخدم حديثة وسهلة الاستخدام\nتصميم عصري يضمن تجربة تعليمية سلسة ومريحة لجميع المستخدمين.\nلوحة تحكم شاملة ومبسطة', metadata={'source': 'https://lxera.net/', 'title': 'Home - Lxera', 'chunk_index': 0, 'uploaded_at': '2025-07-31 12:20:40'}),
 Document(page_content='لوحة تحكم شاملة ومبسطة\nعرض إحصاءات أداء المتدربين بشكل مرئي، بما في ذلك التقدم، النتائج، ومعدلات الإنجاز.\nربط متكامل عبر بوابات الدفع الإلكتروني\nدعم الربط عبر بوابات الدفع الإلكتروني المحلية والعالمية لتسهيل عملية الدفع المالي.\nمس

In [10]:
def _clean(text: str) -> str:
    """Remove common TOC dots, extra whitespace, etc."""
    text = re.sub(r'\.{5,}\s*\d+', '', text)   # dotted TOC lines
    text = re.sub(r'\n{3,}', '\n\n', text)     # huge blank gaps
    return text.strip()

def load_and_chunk_docx(path: str | Path) -> list[Document]:
    docx = DocxDocument(path)
    file_name = Path(path).name
    uploaded  = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # 1️⃣ Collect paragraphs with their nearest heading
    current_heading = "Untitled"
    paragraphs      = []
    for para in docx.paragraphs:
        style = para.style.name
        text  = para.text.strip()
        if not text:
            continue

        if style.startswith("Heading"):          # e.g., 'Heading 1'
            current_heading = text
            continue

        paragraphs.append({"heading": current_heading, "text": text})

    # 2️⃣ Group paragraphs by heading to reduce splitter fragmentation
    grouped = {}
    for p in paragraphs:
        grouped.setdefault(p["heading"], []).append(p["text"])

    docs = []
    for heading, para_list in grouped.items():
        combined = _clean("\n".join(para_list))
        if not combined:
            continue

        # 3️⃣ Split into semantic chunks
        chunks = splitter.split_text(combined)
        for idx, chunk in enumerate(chunks):
            meta = {
                "source": file_name,
                "section": heading,
                "chunk_index": idx,
                "uploaded_at": uploaded,
            }
            docs.append(Document(page_content=chunk, metadata=meta))

    return docs

In [27]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.docstore.document import Document
from tqdm import tqdm

MAX_TOKENS_PER_BATCH = 280000  # Keep some headroom from 300k

def estimate_tokens(text: str) -> int:
    # Very rough approximation: 1 token ≈ 4 characters in English
    return int(len(text) / 4)

def batch_documents(documents, max_tokens=MAX_TOKENS_PER_BATCH):
    batch, token_count = [], 0
    for chunk in documents:
        doc_tokens = estimate_tokens(chunk.page_content)
        if token_count + doc_tokens > max_tokens:
            yield batch
            batch, token_count = [], 0
        batch.append(chunk)
        token_count += doc_tokens
    if batch:
        yield batch

def save_faiss_index():
    word_docs = []
    if os.path.exists("faiss.idx"):
        print("FAISS index already exists, skipping creation.")
        return

    print("Creating FAISS index…")
    chunks = []

    # 1. Load local .docx files
    training = "training"
    for fname in os.listdir(training):
        if fname.startswith("."):
            continue
        word_docs += load_and_chunk_docx(os.path.join(training, fname))

    chunks.extend(splitter.split_documents(word_docs))

    # # 2. Load from live URLs
    # for url in urls:
    #     try:
    #         docs = load_site(url)
    #         chunks.extend(splitter.split_documents(docs))
    #     except Exception as e:
    #         print(f"[WARN] {url} skipped – {e}")

    # print(f"{len(chunks)} chunks total")

    # 3. Create embeddings in batches
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    all_vectors = None

    for batch in tqdm(batch_documents(chunks)):
        db = FAISS.from_documents(batch, embeddings)
        if all_vectors is None:
            all_vectors = db
        else:
            all_vectors.merge_from(db)

    all_vectors.save_local("faiss.idx")
    print("FAISS index created and saved.")


In [26]:
from langchain_community.vectorstores.faiss import FAISS
from langchain_core.documents import Document
import faiss
import numpy as np

def save_faiss_index():
    if os.path.exists("faiss.idx"):
        print("FAISS index already exists, skipping creation.")
        return

    print("Creating FAISS index…")
    
    word_docs = []
    chunks = []

    # Load .docx files
    training = "training"
    for fname in os.listdir(training):
        if fname.startswith("."): continue
        word_docs += load_and_chunk_docx(os.path.join(training, fname))
        chunks.extend(splitter.split_documents(word_docs))

    # Load URLs
    for url in urls:
        try:
            docs = load_site(url)
            chunks.extend(splitter.split_documents(docs))
        except Exception as e:
            print(f"[WARN] {url} skipped – {e}")

    print(f"{len(chunks)} chunks total")

    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    
    vector_chunks = []
    metadatas = []
    docs = []

    for chunk in tqdm(chunks):
        try:
            vector = embeddings.embed_query(chunk.page_content)
            vector_chunks.append(vector)
            metadatas.append(chunk.metadata)
            docs.append(Document(page_content=chunk.page_content, metadata=chunk.metadata))
        except Exception as e:
            print(f"[ERROR] Skipped chunk – {e}")

    text_embedding_pairs = [
        (doc.page_content, vec)
        for doc, vec in zip(docs, vector_chunks)
    ]

    db = FAISS.from_embeddings(text_embedding_pairs, embeddings)

    db.save_local("faiss.idx")

    print("✅ FAISS index created and saved.")
    print({"total_vectors": len(vector_chunks)})


In [27]:
save_faiss_index()

Creating FAISS index…
1261 chunks total


100%|██████████| 1261/1261 [06:55<00:00,  3.04it/s]

✅ FAISS index created and saved.
{'total_vectors': 1261}
